In [2]:
import numpy as np
import pandas as pd
import time
from scipy.stats import norm
from scipy.special import logsumexp

# Set random seed for reproducibility
np.random.seed(42)

# ============================================================================
# SYNTHETIC DATA GENERATION (Based on Table 2 from the paper)
# ============================================================================

def generate_synthetic_prices(n_periods=252, n_regimes=3, spillover=0.0):
    """Generate synthetic natural gas prices with regime-switching dynamics"""
    
    # Regime parameters from Table 2
    regime_params = {
        1: {'kappa': 2.0, 'theta': 20, 'sigma': 0.2, 'lambda_j': 0.1, 'mu_j': 0.0, 'sigma_j': 0.1},
        2: {'kappa': 1.0, 'theta': 25, 'sigma': 0.4, 'lambda_j': 0.3, 'mu_j': 0.1, 'sigma_j': 0.2},
        3: {'kappa': 0.5, 'theta': 40, 'sigma': 0.6, 'lambda_j': 0.5, 'mu_j': 0.2, 'sigma_j': 0.3}
    }
    
    # Transition matrix from Table 3
    Q = np.array([
        [0.94, 0.04, 0.02],
        [0.06, 0.88, 0.06],
        [0.04, 0.06, 0.90]
    ])
    
    # Adjust parameters for spillover effects
    if spillover > 0:
        for k in regime_params:
            regime_params[k]['sigma'] *= (1 + spillover)
            regime_params[k]['lambda_j'] *= (1 + 0.6 * spillover)
    
    # Initialize
    prices = np.zeros(n_periods)
    regimes = np.zeros(n_periods, dtype=int)
    
    # Starting values
    current_regime = np.random.choice(n_regimes)
    prices[0] = regime_params[current_regime + 1]['theta']
    regimes[0] = current_regime
    
    dt = 1/252  # Daily time step
    
    # Generate price path
    for t in range(1, n_periods):
        # Regime transition
        current_regime = np.random.choice(n_regimes, p=Q[current_regime])
        regimes[t] = current_regime
        
        # Get current regime parameters
        params = regime_params[current_regime + 1]
        
        # Mean-reverting dynamics with jumps
        drift = params['kappa'] * (params['theta'] - np.log(prices[t-1]))
        diffusion = params['sigma'] * np.sqrt(dt) * np.random.randn()
        
        # Jump component
        jump = 0
        if np.random.rand() < params['lambda_j'] * dt:
            jump = np.random.normal(params['mu_j'], params['sigma_j'])
        
        # Update price
        log_price = np.log(prices[t-1]) + drift * dt + diffusion + jump
        prices[t] = np.exp(log_price)
        
        # Bound prices for numerical stability
        prices[t] = np.clip(prices[t], 0.1, 1000)
    
    return prices, regimes

# ============================================================================
# STORAGE VALUATION FUNCTIONS
# ============================================================================

def calculate_storage_npv_single_regime(prices, storage_params):
    """Calculate NPV using single-regime model (baseline)"""
    
    start_time = time.time()
    
    # Storage parameters
    max_inventory = storage_params['max_inventory']
    max_injection = storage_params['max_injection']
    max_withdrawal = storage_params['max_withdrawal']
    injection_cost = storage_params['injection_cost']
    withdrawal_cost = storage_params['withdrawal_cost']
    discount_rate = storage_params['discount_rate']
    
    # Discretization
    n_periods = len(prices)
    n_inventory = 20  # Inventory grid points
    inventory_grid = np.linspace(0, max_inventory, n_inventory)
    
    # Initialize value function
    V = np.zeros((n_periods + 1, n_inventory))
    
    # Terminal condition: liquidation value
    for i, inv in enumerate(inventory_grid):
        V[-1, i] = max(0, prices[-1] * inv - withdrawal_cost * inv)
    
    # Backward induction
    dt = 1/252
    discount = np.exp(-discount_rate * dt)
    
    for t in range(n_periods - 1, -1, -1):
        price = prices[t]
        
        for i, inv in enumerate(inventory_grid):
            best_value = -np.inf
            
            # Consider all feasible actions
            actions = []
            
            # Withdrawal
            if inv > 0:
                max_w = min(inv, max_withdrawal * dt)
                actions.extend(np.linspace(-max_w, 0, 5))
            
            # Do nothing
            actions.append(0)
            
            # Injection
            if inv < max_inventory:
                max_i = min(max_inventory - inv, max_injection * dt)
                actions.extend(np.linspace(0, max_i, 5))
            
            for action in actions:
                if action < 0:  # Withdrawal
                    immediate = -action * (price - withdrawal_cost)
                elif action > 0:  # Injection
                    immediate = action * (-price - injection_cost)
                else:  # Hold
                    immediate = 0
                
                # Next inventory
                next_inv = inv + action
                next_inv = np.clip(next_inv, 0, max_inventory)
                
                # Interpolate future value
                next_idx = np.searchsorted(inventory_grid, next_inv)
                if next_idx == 0:
                    future = V[t + 1, 0]
                elif next_idx >= n_inventory:
                    future = V[t + 1, -1]
                else:
                    # Linear interpolation
                    alpha = (next_inv - inventory_grid[next_idx - 1]) / \
                            (inventory_grid[next_idx] - inventory_grid[next_idx - 1])
                    future = (1 - alpha) * V[t + 1, next_idx - 1] + alpha * V[t + 1, next_idx]
                
                value = immediate + discount * future
                best_value = max(best_value, value)
            
            V[t, i] = best_value
    
    computation_time = time.time() - start_time
    
    # NPV is the value at t=0 with zero inventory
    npv = V[0, 0]
    
    return npv, computation_time

def calculate_storage_npv_multi_regime(prices, regimes, n_regimes, spillover=0.0, storage_params=None):
    """Calculate NPV using multi-regime model"""
    
    start_time = time.time()
    
    # Storage parameters
    max_inventory = storage_params['max_inventory']
    max_injection = storage_params['max_injection']
    max_withdrawal = storage_params['max_withdrawal']
    injection_cost = storage_params['injection_cost']
    withdrawal_cost = storage_params['withdrawal_cost']
    discount_rate = storage_params['discount_rate']
    
    # Transition matrix
    Q = np.array([
        [0.94, 0.04, 0.02],
        [0.06, 0.88, 0.06],
        [0.04, 0.06, 0.90]
    ])
    
    # Adjust for spillover effects
    if spillover > 0:
        # Spillover increases regime persistence during stress
        Q[2, 2] = min(0.95, Q[2, 2] + 0.05 * spillover)
        # Renormalize
        Q[2, :2] = (1 - Q[2, 2]) * Q[2, :2] / Q[2, :2].sum()
    
    # Discretization
    n_periods = len(prices)
    n_inventory = 20
    inventory_grid = np.linspace(0, max_inventory, n_inventory)
    
    # Initialize value function for each regime
    V = np.zeros((n_periods + 1, n_inventory, n_regimes))
    
    # Terminal condition
    for k in range(n_regimes):
        for i, inv in enumerate(inventory_grid):
            V[-1, i, k] = max(0, prices[-1] * inv - withdrawal_cost * inv)
    
    # Backward induction
    dt = 1/252
    discount = np.exp(-discount_rate * dt)
    
    for t in range(n_periods - 1, -1, -1):
        price = prices[t]
        current_regime = regimes[t]
        
        for k in range(n_regimes):
            for i, inv in enumerate(inventory_grid):
                best_value = -np.inf
                
                # Consider feasible actions
                actions = []
                
                # Withdrawal
                if inv > 0:
                    max_w = min(inv, max_withdrawal * dt)
                    actions.extend(np.linspace(-max_w, 0, 5))
                
                # Hold
                actions.append(0)
                
                # Injection
                if inv < max_inventory:
                    max_i = min(max_inventory - inv, max_injection * dt)
                    actions.extend(np.linspace(0, max_i, 5))
                
                for action in actions:
                    if action < 0:  # Withdrawal
                        immediate = -action * (price - withdrawal_cost)
                    elif action > 0:  # Injection
                        immediate = action * (-price - injection_cost)
                    else:
                        immediate = 0
                    
                    # Next inventory
                    next_inv = inv + action
                    next_inv = np.clip(next_inv, 0, max_inventory)
                    
                    # Expected future value across regimes
                    future = 0
                    for k_next in range(n_regimes):
                        # Interpolate
                        next_idx = np.searchsorted(inventory_grid, next_inv)
                        if next_idx == 0:
                            future_k = V[t + 1, 0, k_next]
                        elif next_idx >= n_inventory:
                            future_k = V[t + 1, -1, k_next]
                        else:
                            alpha = (next_inv - inventory_grid[next_idx - 1]) / \
                                    (inventory_grid[next_idx] - inventory_grid[next_idx - 1])
                            future_k = (1 - alpha) * V[t + 1, next_idx - 1, k_next] + \
                                       alpha * V[t + 1, next_idx, k_next]
                        
                        future += Q[k, k_next] * future_k
                    
                    value = immediate + discount * future
                    best_value = max(best_value, value)
                
                V[t, i, k] = best_value
    
    computation_time = time.time() - start_time
    
    # NPV starting from current regime with zero inventory
    npv = V[0, 0, regimes[0]]
    
    return npv, computation_time

# ============================================================================
# MAIN ANALYSIS - TABLE 4 GENERATION
# ============================================================================

def generate_table_4():
    """Generate Table 4: NPV Comparison"""
    
    print("Generating Table 4: NPV Comparison")
    print("=" * 60)
    
    # Storage facility parameters
    storage_params = {
        'max_inventory': 1000,  # MWh
        'max_injection': 50,    # MWh/day
        'max_withdrawal': 100,  # MWh/day
        'injection_cost': 0.5,  # €/MWh
        'withdrawal_cost': 0.5, # €/MWh
        'discount_rate': 0.05   # Annual
    }
    
    # Generate synthetic data
    n_simulations = 10  # Multiple simulations for robustness
    results = []
    
    for sim in range(n_simulations):
        print(f"\nSimulation {sim + 1}/{n_simulations}")
        
        # Generate prices without spillover
        prices_no_spill, regimes_no_spill = generate_synthetic_prices(
            n_periods=252, n_regimes=3, spillover=0.0
        )
        
        # Generate prices with spillover
        prices_spill, regimes_spill = generate_synthetic_prices(
            n_periods=252, n_regimes=3, spillover=0.5
        )
        
        # Single-regime model (baseline)
        npv_single, time_single = calculate_storage_npv_single_regime(
            prices_no_spill, storage_params
        )
        
        # Multi-regime without spillover
        npv_multi_no_spill, time_multi_no_spill = calculate_storage_npv_multi_regime(
            prices_no_spill, regimes_no_spill, n_regimes=3, 
            spillover=0.0, storage_params=storage_params
        )
        
        # Multi-regime with spillover
        npv_multi_spill, time_multi_spill = calculate_storage_npv_multi_regime(
            prices_spill, regimes_spill, n_regimes=3, 
            spillover=0.5, storage_params=storage_params
        )
        
        results.append({
            'single': npv_single,
            'multi_no_spill': npv_multi_no_spill,
            'multi_spill': npv_multi_spill,
            'time_single': time_single,
            'time_multi_no_spill': time_multi_no_spill,
            'time_multi_spill': time_multi_spill
        })
    
    # Calculate averages
    results_df = pd.DataFrame(results)
    
    # Normalize to baseline = 100
    baseline_npv = results_df['single'].mean()
    
    # Create Table 4
    table_4 = pd.DataFrame([
        {
            'Model': 'Single-regime',
            'NPV (€/MWh)': 100.0,
            'Improvement': 'Baseline',
            'Computation Time': f"{results_df['time_single'].mean():.1f} seconds"
        },
        {
            'Model': 'Multi-regime (no spillover)',
            'NPV (€/MWh)': 100 * results_df['multi_no_spill'].mean() / baseline_npv,
            'Improvement': f"+{100 * (results_df['multi_no_spill'].mean() / baseline_npv - 1):.1f}%",
            'Computation Time': f"{results_df['time_multi_no_spill'].mean():.1f} seconds"
        },
        {
            'Model': 'Multi-regime (with spillover)',
            'NPV (€/MWh)': 100 * results_df['multi_spill'].mean() / baseline_npv,
            'Improvement': f"+{100 * (results_df['multi_spill'].mean() / baseline_npv - 1):.1f}%",
            'Computation Time': f"{results_df['time_multi_spill'].mean():.1f} seconds"
        }
    ])
    
    # Display results
    print("\n" + "=" * 60)
    print("TABLE 4: NPV COMPARISON")
    print("=" * 60)
    print(table_4.to_string(index=False))
    
    # Statistical significance testing (bootstrap)
    print("\n" + "=" * 60)
    print("STATISTICAL SIGNIFICANCE")
    print("=" * 60)
    
    improvement_no_spill = 100 * (results_df['multi_no_spill'] / results_df['single'] - 1)
    improvement_spill = 100 * (results_df['multi_spill'] / results_df['single'] - 1)
    
    print(f"Multi-regime (no spillover) improvement: {improvement_no_spill.mean():.1f}% " +
          f"[95% CI: {improvement_no_spill.quantile(0.025):.1f}%, " +
          f"{improvement_no_spill.quantile(0.975):.1f}%]")
    
    print(f"Multi-regime (with spillover) improvement: {improvement_spill.mean():.1f}% " +
          f"[95% CI: {improvement_spill.quantile(0.025):.1f}%, " +
          f"{improvement_spill.quantile(0.975):.1f}%]")
    
    return table_4, results_df

# Run the analysis
if __name__ == "__main__":
    table_4, detailed_results = generate_table_4()
    
    print("\n" + "=" * 60)
    print("Note: These results are from a simplified synthetic demonstration")
    print("with coarse discretization. Actual improvements depend on market")
    print("conditions and storage characteristics.")
    print("=" * 60)

Generating Table 4: NPV Comparison

Simulation 1/10

Simulation 2/10

Simulation 3/10

Simulation 4/10

Simulation 5/10

Simulation 6/10

Simulation 7/10

Simulation 8/10

Simulation 9/10

Simulation 10/10

TABLE 4: NPV COMPARISON
                        Model  NPV (€/MWh) Improvement Computation Time
                Single-regime   100.000000    Baseline      1.7 seconds
  Multi-regime (no spillover)   100.000000      +-0.0%      8.3 seconds
Multi-regime (with spillover)   112.234613      +12.2%      8.2 seconds

STATISTICAL SIGNIFICANCE
Multi-regime (no spillover) improvement: -0.0% [95% CI: -0.0%, -0.0%]
Multi-regime (with spillover) improvement: 13.3% [95% CI: -3.2%, 34.6%]

Note: These results are from a simplified synthetic demonstration
with coarse discretization. Actual improvements depend on market
conditions and storage characteristics.
